## Find a potentially important facility for its health impact

Author : Yunha Lee

Date : Apr 7, 2025

The goal of this script is to find a facility with modified emissions by CCS technology that might contirbute to large AQ and health impact. I am going to use the INMAP or BenMAP health impact output and CCS emissions to determine an potentially important facility. 

Update : Trying to find the facility from high emitter is not easy because several small emitters can colocated with them, and it is unclear which facilities are responsible for the large health impact. Instead of this approach, I am going to find the large health impact areas (using buffer) and then examine the emitters in the buffer. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable


def plot_buffer_and_base(gdf_base, base_colname, gdf_buffer, buffer_colname, number_of_buffer=None, buffer_plot_first=None):
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib.colors as mcolors
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    if number_of_buffer:
        # Subset the top facilities
        gdf_buffer = gdf_buffer.sort_values(by=buffer_colname, ascending=False)
        gdf_buffer = gdf_buffer[:number_of_buffer]
        
        vmin = gdf_buffer[buffer_colname].min()
        vmax = gdf_buffer[buffer_colname].max()
        
        # Define the number of discrete bins
        color_map = plt.cm.spring_r  # Choose a sequential colormap
        bounds = np.linspace(vmin, vmax, number_of_buffer + 1)  # Define color step boundaries
        norm = mcolors.BoundaryNorm(bounds, color_map.N)  # Create a discrete colormap
    else:
        color_map = plt.cm.spring_r
        norm = None
    
    # Normalize the buffer values for better scaling of marker size and color
    buffer_values = gdf_buffer[buffer_colname]
    size_scaled = (buffer_values - buffer_values.min()) / (buffer_values.max() - buffer_values.min()) * 1000 + 50  # scale for visibility
    
    # Plotting the data
    fig, ax = plt.subplots(figsize=(5, 5))
    
    divider = make_axes_locatable(ax)
    cax_right = divider.append_axes("right", size="5%", pad=0.1)
    cax_left = divider.append_axes("bottom", size="5%", pad=0.2)
    
    if not buffer_plot_first:
        # Plot base first
        base_plot = gdf_base.plot(
            column=base_colname, 
            ax=ax, 
            cmap='Blues', 
            alpha=0.9,
            legend=False  # We'll create the colorbar separately
        )
        
        # Create colorbar for base
        base_sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(
            vmin=gdf_base[base_colname].min(), 
            vmax=gdf_base[base_colname].max()
        ))
        base_sm.set_array([])
        base_cbar = fig.colorbar(base_sm, cax=cax_right)
        base_cbar.set_label(base_colname, rotation=270, labelpad=20)
        
        # Plot buffer points
        buffer_plot = gdf_buffer.plot(
            ax=ax,
            column=buffer_colname,
            cmap=color_map, 
            norm=norm,
            markersize=size_scaled,
            alpha=0.7,
            legend=False  # We'll create the colorbar separately
        )
        
        # Create colorbar for buffer
        buffer_sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
        buffer_sm.set_array([])
        buffer_cbar = fig.colorbar(buffer_sm, cax=cax_left,  orientation="horizontal")
        buffer_cbar.set_label(buffer_colname, labelpad=20)
        
    else:
        # Plot buffer points first
        buffer_plot = gdf_buffer.plot(
            ax=ax,
            column=buffer_colname,
            cmap=color_map, 
            norm=norm,
            markersize=size_scaled,
            alpha=0.5,
            legend=False  # We'll create the colorbar separately
        )
        
        # Create colorbar for buffer
        buffer_sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
        buffer_sm.set_array([])
        buffer_cbar = fig.colorbar(buffer_sm, cax=cax_left , orientation="horizontal")
        buffer_cbar.set_label(buffer_colname, labelpad=20)
        
        # Plot base 
        base_plot = gdf_base.plot(
            column=base_colname, 
            ax=ax, 
            cmap='Blues', 
            alpha=0.9,
            legend=False  # We'll create the colorbar separately
        )
        
        # Create colorbar for base
        base_sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(
            vmin=gdf_base[base_colname].min(), 
            vmax=gdf_base[base_colname].max()
        ))
        base_sm.set_array([])
        base_cbar = fig.colorbar(base_sm, cax=cax_right)
        base_cbar.set_label(base_colname, rotation=270, labelpad=20)
    
    # Add a proper title with better positioning
    plt.suptitle(f'{buffer_colname} and {base_colname}', y=0.98, fontsize=14)
    
    # Remove axis ticks but keep the extent visible
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    
    plt.tight_layout()
    plt.show()
    
    return fig, ax

In [ ]:

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import pandas as pd

def find_top_3_facilities (gdf_health, gdf_emis, buffer_distance_list,  species_list, output_dir):
    # sort health output
    gdf_health = gdf_health.sort_values(by='TotalPopD', ascending=False)
    gdf_health_top = gdf_health[:10]

    gdf_health_top = gdf_health_top[['geometry','TotalPopD','FIPS']]

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for buffer_distance in buffer_distance_list:
        buffer_mortality_totals = []
        buffer_totals = []
        gdf_buffers_data = {
            'geometry': [],
            f'total_mortality_{buffer_distance}': [],
        }

        species_total = {s: [] for s in species_list}
        species_top3 = {s: [[], [], [], []] for s in species_list}  # top1, top2, top3, others
        species_top3_ids = {s: [[], [], []] for s in species_list}  # top1_id, top2_id, top3_id
        species_top3_contrib = {s: [] for s in species_list}

        for idx, grid in gdf_health_top.iterrows():
            grid_buffer = grid.geometry.buffer(buffer_distance)
            buffer_totals.append(grid_buffer)

            # Mortality
            intersecting_areas = gdf_health[gdf_health.intersects(grid_buffer)]
            total_mortality = intersecting_areas['TotalPopD'].abs().sum()
            buffer_mortality_totals.append(total_mortality)

            facilities_in_buffer = gdf_emis[gdf_emis.intersects(grid_buffer)]

            print(facilities_in_buffer)

            # Process species
            for sp in species_list:
                total_emis = facilities_in_buffer[sp].sum()
                species_total[sp].append(total_emis)

                top3_emitters = facilities_in_buffer.nlargest(3, sp)
                top_vals = top3_emitters[sp].tolist()
                top_ids = top3_emitters['EIS_ID'].tolist()

                while len(top_vals) < 3:
                    top_vals.append(0.0)
                while len(top_ids) < 3:
                    top_ids.append(None)

                others = total_emis - sum(top_vals)
                for i in range(3):
                    species_top3[sp][i].append(top_vals[i])
                    species_top3_ids[sp][i].append(top_ids[i])
                species_top3[sp][3].append(others)

                species_top3_contrib[sp].append(top_vals + [others])

        # Create GeoDataFrame
        gdf_buffers = gpd.GeoDataFrame(geometry=buffer_totals)
        gdf_buffers[f'total_mortality_{buffer_distance}'] = buffer_mortality_totals

        for sp in species_list:
            gdf_buffers[f'total_{sp}_{buffer_distance}'] = species_total[sp]
            gdf_buffers[f'{sp}_top1'] = species_top3[sp][0]
            gdf_buffers[f'{sp}_top2'] = species_top3[sp][1]
            gdf_buffers[f'{sp}_top3'] = species_top3[sp][2]
            gdf_buffers[f'{sp}_others'] = species_top3[sp][3]

            gdf_buffers[f'{sp}_top1_ID'] = species_top3_ids[sp][0]
            gdf_buffers[f'{sp}_top2_ID'] = species_top3_ids[sp][1]
            gdf_buffers[f'{sp}_top3_ID'] = species_top3_ids[sp][2]


        gdf_buffers = gdf_buffers.sort_values(by=f'total_mortality_{buffer_distance}', ascending=True)
        gdf_buffers['FIPS'] = gdf_health_top['FIPS'].values
        gdf_health_top[f'total_mortality_{buffer_distance}'] = buffer_mortality_totals
    
        # Save buffer layer
        gdf_buffers.to_file(os.path.join(output_dir, f"gdf_buffers_{buffer_distance}.shp"))

        

        # Plot mortality vs stacked top 3 emissions per species using gdf_buffers
        for sp in species_list:

            plot_buffer_and_base(gdf_health, 'TotalPopD', gdf_buffers, f'{sp}_top1', number_of_buffer=5, buffer_plot_first=False)

            # Extract columns from gdf_buffers
            top1 = gdf_buffers[f'{sp}_top1']
            top2 = gdf_buffers[f'{sp}_top2']
            top3 = gdf_buffers[f'{sp}_top3']
            rest = gdf_buffers[f'{sp}_others']
            mortality = gdf_buffers[[f'total_mortality_{buffer_distance}', 'FIPS']]

            # Extract EIS_IDs
            top1_ids = gdf_buffers[f'{sp}_top1_ID']
            top2_ids = gdf_buffers[f'{sp}_top2_ID']
            top3_ids = gdf_buffers[f'{sp}_top3_ID']

            ids = gdf_buffers['FIPS'] 


            # Create DataFrame for stacked bar (index = 'ID')
            df_stacked = pd.DataFrame({
                'Top1': top1,
                'Top2': top2,
                'Top3': top3,
                'The rest': rest,
                'FIPS': ids
            })

            mortality.set_index('FIPS', inplace=True)
            df_stacked.set_index('FIPS', inplace=True)

            #print("gdf_buffers", gdf_buffers.head())
            #print("df_stack", df_stacked.head())

            fig, ax1 = plt.subplots(figsize=(10, 5))
            ax2 = ax1.twinx()

            # Plot stacked top3 emissions
            df_stacked.plot(kind='bar', stacked=True, ax=ax1, colormap='autumn', width=0.3, position=-0.5, alpha = 0.5)
            
            # Plot mortality on second y-axis
            mortality.plot(kind='bar', ax=ax2, color='tab:blue', width=0.3, label='Total Mortality',position=0.5, alpha = 0.5)

            # Add EIS_ID labels on top of each top bar
            for i in range(len(df_stacked)):
                y_offset_top1 = top1.iloc[i]
                y_offset_top2 = y_offset_top1 + top2.iloc[i]
                y_offset_top3 = y_offset_top2 + top3.iloc[i]

                # Label Top1 bar
                if y_offset_top1 > 0:
                    ax1.text(i, y_offset_top1 / 2, int(top1_ids.iloc[i]), ha='center', va='center', fontsize=8, rotation=0)

                # Label Top2 bar
                if top2.iloc[i] > 0:
                    ax1.text(i, y_offset_top1 + top2.iloc[i] / 2, int(top2_ids.iloc[i]), ha='center', va='center', fontsize=8, rotation=0)

                # Label Top3 bar
                if top3.iloc[i] > 0:
                    ax1.text(i, y_offset_top2 + top3.iloc[i] / 2, int(top3_ids.iloc[i]), ha='center', va='center', fontsize=8, rotation=0)


            ax1.set_ylabel(f"{sp} Emissions")
            ax2.set_ylabel("Total Mortality")
            ax1.set_xlabel("FIPS")
            ax1.set_title(f"{sp} Top 3 Facility Emissions vs. Mortality in {buffer_distance/1000:.0f} km Buffers")

            ax1.legend(loc='upper left', title='Top Emitters')
            ax2.legend(loc='upper center')

            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f"compare_stacked_{sp}_{buffer_distance}m.png"))
            plt.close()

    return gdf_health_top




# Pre-step : Read CCS emission and INMAP output

In [ ]:
import geopandas as gpd

# read base and sens emission scenarios
gdf_emis = gpd.read_file(
    '/Users/yunhalee/Documents/LOCAETA/CS_emissions/Colorado_point_CCS_reduced_emis_wo_NH3_VOC.shp') # ')

# Reset index to ensure proper comparison
gdf_emis.reset_index(drop=True, inplace=True)

# Reproject to the appropriate UTM zone for accurate distance measurements
if gdf_emis.crs is None:
    gdf_emis.set_crs(epsg=32618, inplace=True)
else:
    gdf_emis.to_crs(epsg=32618, inplace=True)

# drop unnecessary columns
gdf_emis.drop(columns = ['SCC', 'height', 'diam', 'velocity','temp'], inplace=True)

# Sum emissions over same EIS_ID
emissions_sum = gdf_emis.groupby('EIS_ID')[['PM2_5', 'SOx', 'NOx', 'NH3', 'VOC','PM2_5_old', 'SOx_old', 'NH3_new', 'NOx_old', 'VOC_new']].sum()

# Get first occurrence of other attributes (including geometry) per EIS_ID
metadata = gdf_emis.groupby('EIS_ID').first()

# Merge the summed emissions back into the metadata
gdf_emis_agg = metadata.drop(columns=['PM2_5', 'SOx', 'NOx', 'NH3', 'VOC','PM2_5_old', 'SOx_old', 'NOx_old', 'NH3_new', 'VOC_new']).join(emissions_sum)

# Convert to GeoDataFrame to retain spatial features
gdf_emis = gpd.GeoDataFrame(gdf_emis_agg, geometry='geometry')

gdf_emis.reset_index(inplace=True)

# Reproject to the appropriate UTM zone for accurate distance measurements
if gdf_emis.crs is None:
    gdf_emis.set_crs(epsg=32618, inplace=True)
else:
    gdf_emis.to_crs(epsg=32618, inplace=True)
    
print(gdf_emis.head())


In [ ]:

# check the total emissions
col_list = ['PM2_5', 'SOx', 'NOx', 'NH3', 'VOC','PM2_5_old', 'SOx_old', 'NOx_old', 'NH3_new', 'VOC_new']
column_sums = gdf_emis[col_list].sum()
column_sums

# Rename and Compute the net emission changes for three key species
for col in ['PM2_5', 'SOx', 'NOx']:
    gdf_emis[col+'_new']= gdf_emis[col]
    gdf_emis[col]= gdf_emis[col+'_old'] - gdf_emis[col]

column_sums = gdf_emis[col_list].sum()
column_sums

In [ ]:
def plot_scatter_with_1to1_line (gdf_emis, x_name, y_name, title_name):


    ax = gdf_emis.plot.scatter(x=x_name, y = y_name, title=title_name)
    # Get the current limits of the axes
    xlims = ax.get_xlim()
    ylims = ax.get_ylim()

    # Use the min and max of both limits to define the 1:1 line
    limit_min = min(xlims[0], ylims[0])
    limit_max = max(xlims[1], ylims[1])

    # Plot the 1:1 line
    ax.plot([limit_min, limit_max], [limit_min, limit_max], color='red', linestyle='--', label='1:1 Line')

    # Reset the limits to include the 1:1 line
    ax.set_xlim(limit_min, limit_max)
    ax.set_ylim(limit_min, limit_max)

    if y_name == 'PM2_5':
        plt.ylabel('PM2_5 emission change')

    # Add legend
    ax.legend()

    # Display the plot
    plt.show()

plot_scatter_with_1to1_line(gdf_emis, x_name='PM2_5_new', y_name = 'PM2_5', title_name='Relationship between final CCS emissions and CCS emission changes' )

plot_scatter_with_1to1_line(gdf_emis, x_name='PM2_5_old', y_name= 'PM2_5', title_name='Relationship between NEI emissions and CCS emission changes')

plot_scatter_with_1to1_line(gdf_emis, x_name='PM2_5_old', y_name = 'PM2_5_new', title_name='Relationship between NEI emissions and CCS emission')

In [ ]:
import os, sys

package_path = os.path.abspath('/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/LOCAETA_AQ')
if package_path not in sys.path:
    sys.path.append(package_path)

import inmap_analysis

inmap_run_dir = '/Users/yunhalee/Documents/LOCAETA/RCM/INMAP/inmap-1.9.6-gridsplit/outputs/'

state_regions = {"CO": '08'}  # {"LA": ['22']} #,'05', "28", "48"]}   # 

# Define pairs of base and sensitivity runs
run_pairs = {
    # 'LA_CCS': {
    #      'base': 'base_nei2020/2020nei_output_run_steady.shp',
    #      'sens': 'LA_CCS/2020nei_output_run_steady.shp'
    #  },
    #  'CO_CCS': {
    #      'base': 'base_nei2020/2020nei_output_run_steady.shp',
    #      'sens': 'CO_CCS/2020nei_output_run_steady.shp'
    #  },
     'CO_CCS_wo_NH3_VOC': {
          'base': 'base_nei2020/2020nei_output_run_steady.shp',
          'sens': 'CO_CCS_wo_NH3_VOC/2020nei_output_run_steady.shp'
}}

for run_name, paths in run_pairs.items():
    gdf_health = inmap_analysis.process_run_pair(run_name, paths, inmap_run_dir)


    # Somehow one grid has larger mortality change than population..
    to_check = gdf_health[(abs(gdf_health['TotalPopD_base']) > abs(gdf_health['TotalPop_base']))]
    print("Rows to be deleted due to wrong mortality:\n", to_check)
    gdf_health = gdf_health.drop(to_check.index)

    if gdf_health.crs is None:
        gdf_health.set_crs(epsg=32618, inplace=True)
    else:
        gdf_health.to_crs(epsg=32618, inplace=True)

    # subset the dataframe
    for state, state_fips in state_regions.items():
        gdf_health = inmap_analysis.subset_state(gdf_health, state_fips)
    
# convert the mortality sign 
gdf_health['TotalPopD'] = gdf_health['TotalPopD'] * -1.

gdf_health.plot(column='TotalPopD', legend=True)
gdf_health.plot(column='TotalPM25', legend=True)

gdf_health.plot(kind = 'scatter', x = 'TotalPopD', y= 'TotalPM25' ) 

# Check TotalPopD and TotalPM25 relationship with county-level INMAP data

In [ ]:

grid_shapefile_path = '/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/grids/County/County.shp'
gdf_grids = gpd.read_file(grid_shapefile_path)

# rename FIPS temporary
gdf_grids = gdf_grids.rename(columns={'FIPS': 'ID'})

print("CRS of inmap:", gdf_health.crs)
print("CRS of gdf_grids before conversion:", gdf_grids.crs)

# match CRS 
gdf_grids =  gdf_grids.to_crs(gdf_health.crs) 

# subset the dataframe
for state, state_fips in state_regions.items():
    gdf_grids = inmap_analysis.subset_state(gdf_grids, state_fips)

In [ ]:
import geopandas as gpd
import pandas as pd

def area_weighted_interpolation(
    source_gdf,  # e.g., gdf_health
    target_gdf,  # e.g., gdf_grids or gdf_county
    value_columns,  # list of columns to interpolate, e.g., ['TotalPopD']
    target_id_col='ID'  # ID column in the target_gdf
):
    # Ensure CRS matches
    source_gdf = source_gdf.to_crs(target_gdf.crs)

    # Add original area to source
    source_gdf['orig_area'] = source_gdf.geometry.area

    # Subset only necessary columns
    source_cols = ['geometry', 'orig_area'] + value_columns
    target_cols = ['geometry', target_id_col]

    # Perform spatial intersection
    intersected = gpd.overlay(source_gdf[source_cols], target_gdf[target_cols], how='intersection')

    # Calculate area of intersection and area fraction
    intersected['intersect_area'] = intersected.geometry.area
    intersected['area_frac'] = intersected['intersect_area'] / intersected['orig_area']

    # Multiply each value column by the area fraction
    for col in value_columns:
        intersected[f'weighted_{col}'] = intersected[col] * intersected['area_frac']

    # Aggregate weighted values by the target ID
    agg_dict = {f'weighted_{col}': 'sum' for col in value_columns}
    result_df = intersected.groupby(target_id_col).agg(agg_dict).reset_index()

    # Rename back to original names
    result_df = result_df.rename(columns={f'weighted_{col}': col for col in value_columns})


    # Merge back with geometry from target_gdf
    result_gdf = target_gdf[[target_id_col, 'geometry']].merge(result_df, on=target_id_col, how='left')

    # Return as GeoDataFrame
    return gpd.GeoDataFrame(result_gdf, geometry='geometry', crs=target_gdf.crs)

inmap_column_list = ['PNH4', 'PSO4', 'PNO3', 'PrimPM25','TotalPM25','TotalPopD']
gdf_county = area_weighted_interpolation(gdf_health, gdf_grids, inmap_column_list, target_id_col='ID')

gdf_county.rename(columns= {'ID':'FIPS'}, inplace=True)

output_dir = f"/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/find_facility/CO_runs/"
buffer_out_path = os.path.join(output_dir, f"inmap_output_county_level.shp")
gdf_county.to_file(buffer_out_path)

In [ ]:
print(len(gdf_county['FIPS'].duplicated()), len(gdf_county))

if True in gdf_county['FIPS'].duplicated():
    print ("yes")

else:
    print("no")

In [ ]:
import geopandas as gpd
import pandas as pd

def aggregate_emissions_to_counties(gdf_emis, gdf_grids, species_cols, county_id_col='ID'):
    """
    Spatially join point emissions to counties and aggregate emissions by county and species.

    Parameters:
    - gdf_emis: GeoDataFrame with point source emissions.
    - gdf_grids: GeoDataFrame with county polygons and unique ID column.
    - species_cols: list of column names (e.g., ['SOx', 'NOx', 'PM2_5']) to aggregate.
    - county_id_col: name of the column in gdf_grids representing county ID.

    Returns:
    - GeoDataFrame with total emissions by species for each county.
    """

    # Ensure CRS matches
    gdf_emis = gdf_emis.to_crs(gdf_grids.crs)

    # Spatial join: assign each point emission to a county
    joined = gpd.sjoin(gdf_emis, gdf_grids[[county_id_col, 'geometry']], how='left', predicate='within')

    # Aggregate emissions by county ID
    agg_emis = joined.groupby(county_id_col)[species_cols].sum().reset_index()

    # Merge with county geometry for output GeoDataFrame
    result = gdf_grids[[county_id_col, 'geometry']].merge(agg_emis, on=county_id_col, how='left')

    return gpd.GeoDataFrame(result, geometry='geometry')


emis_column_list = ['SOx', 'NOx', 'PM2_5']

gdf_emis_county = aggregate_emissions_to_counties(gdf_emis, gdf_grids, emis_column_list, county_id_col='ID')
gdf_emis_county.rename(columns= {'ID':'FIPS'}, inplace=True)

buffer_out_path = os.path.join(output_dir, f"emis_county_level.shp")
gdf_emis_county.to_file(buffer_out_path)

# Base map plot
gdf_emis_county.plot(
    column='PM2_5',
    legend=True,
    alpha = 0.5
)



In [ ]:
# map the top counties with each species emissions

for sp in target_sp_list: 
    top_emis = gdf_emis_county.sort_values(by=sp, ascending=False).head(5).set_index('FIPS')

    plot_map_with_fips(top_emis, column =sp, title =f'{sp} point emissions at Top 5 Counties (by point emissions agg by county)', gdf_emis = gdf_emis, species =  target_sp_list)







In [ ]:
def plot_species_stacked_by_facility_per_species_top_counties(
    gdf_emis,
    gdf_grids,
    species_cols,
    county_id_col='ID',
    top_n_counties=5,
    top_n_facilities=3
):
    import matplotlib.pyplot as plt

    # Ensure CRS match
    gdf_emis = gdf_emis.to_crs(gdf_grids.crs)

    # Spatial join: assign each point emission to a county
    joined = gpd.sjoin(
        gdf_emis,
        gdf_grids[[county_id_col, 'geometry']],
        how='left',
        predicate='within'
    ).dropna(subset=[county_id_col])

    # Assign facility ID if not present
    if 'EIS_ID' not in joined.columns:
        joined['EIS_ID'] = joined.index.astype(str)

    for species in species_cols:
        # 1. Get top counties for this species
        top_counties = (
            joined.groupby(county_id_col)[species]
            .sum()
            .nlargest(top_n_counties)
            .index
        )

        print("checking top counties",top_n_counties)
        print("checking top counties", top_counties)

        for cid in top_counties:
            county_df = joined[joined[county_id_col] == cid].copy()
            plot_data = []

            for sp in species_cols:
                # Group by facility
                df = (
                    county_df.groupby('EIS_ID')[sp]
                    .sum()
                    .reset_index()
                )
                df_sorted = df.sort_values(by=sp, ascending=False)

                # Top facilities for this species
                top_emitters = df_sorted.head(top_n_facilities).copy()
                top_emitters['Label'] = [f"{sp} - {fid}" for fid in top_emitters['EIS_ID']]

                # Sum "Other" facilities
                other_sum = df_sorted[sp].iloc[top_n_facilities:].sum()
                if other_sum > 0:
                    other_row = pd.DataFrame({
                        'EIS_ID': ['Other'],
                        sp: [other_sum],
                        'Label': [f"{sp} - Other"]
                    })
                    combined = pd.concat([top_emitters, other_row], ignore_index=True)
                else:
                    combined = top_emitters

                combined['Species'] = sp
                combined.rename(columns={sp: 'Emissions'}, inplace=True)
                plot_data.append(combined)

            # Combine all species data for this county
            plot_df = pd.concat(plot_data)

            # Pivot to shape for plotting
            pivot_df = plot_df.pivot_table(
                index='Species',
                columns='Label',
                values='Emissions',
                fill_value=0
            )

            # Plot
            ax = pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='tab20')
            ax.set_ylabel('Emissions')
            ax.set_title(f"Top Emitters by {species} in County {cid} (Top {top_n_facilities} per species)")
            ax.legend(title='Species - EIS_ID', bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.tight_layout()
            plt.show()

species_cols = ['SOx', 'NOx', 'PM2_5']
plot_species_stacked_by_facility_per_species_top_counties(gdf_emis, gdf_grids, species_cols)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_map_with_fips(
    gdf,
    column,
    title,
    gdf_emis=None,
    species=None,
    cmap="OrRd",
    top_n=5,
    ax=None
):
    
    species_rotation = {
    'SOx': 0,      
    'NOx': 45,     
    'PM2_5': 90,   
    }
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)

    # Base map plot
    gdf.plot(
        column=column,
        ax=ax,
        cax=cax,
        legend=True,
        cmap=cmap,
        legend_kwds={"label": title},
        alpha = 0.5
    )

    # Add FIPS labels at centroid
    for idx, row in gdf.iterrows():
        centroid = row.geometry.centroid
        ax.text(
            centroid.x, centroid.y,
            str(row.name),  # assumes FIPS is the index
            fontsize=8, ha='center', va='center', color='black'
        )

    # If emission data and species provided
    if gdf_emis is not None and species is not None:
        # Predefined colors for species
        colors = ['blue', 'green', 'purple', 'orange', 'cyan']
        species_colors = {sp: colors[i % len(colors)] for i, sp in enumerate(species)}

        for sp in species:
            if sp not in gdf_emis.columns:
                continue

            top_emitters = gdf_emis.sort_values(by=sp, ascending=False).head(top_n)

            top_emitters.plot(
                ax=ax,
                color=species_colors[sp],
                markersize=50,
                label=f'Top {top_n} {sp} Emitters'
            )

            rotation_angle = species_rotation.get(sp, 0)

            # Label each top emitter with species and rank
            for i, (_, row) in enumerate(top_emitters.iterrows(), 1):
                centroid = row.geometry.centroid
                ax.text(
                    centroid.x,
                    centroid.y,
                    f"{sp[:3]}: #{i}",
                    fontsize=7,
                    color=species_colors[sp],
                    ha='center',
                    rotation=rotation_angle
                )

        ax.legend(title="Top Emitters by Species", fontsize=8)

    ax.set_title(title)
    ax.axis('off')
    plt.tight_layout()
    plt.show()



In [ ]:
# --- Select Top Counties ---
top_pm25 = gdf_county.sort_values(by='TotalPM25', ascending=True).head(5).set_index('FIPS')
top_mortality = gdf_county.sort_values(by='TotalPopD', ascending=False).head(5).set_index('FIPS')

# --- Bar Plot: Top 5 PM2.5 Counties ---
top_pm25.plot(kind='bar', legend=True, figsize=(8, 4))
plt.title("Top 5 Counties by PM2.5")
# Place the legend outside the plot
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
#plt.legend(bbox_to_anchor=(1.0, 1.0))

#plt.ylabel("Total PM2.5")
plt.tight_layout()
plt.show()


target_sp_list = ['SOx', 'NOx','PM2_5']

# --- Map Plot: Mortality at Top PM2.5 Counties ---
plot_map_with_fips(top_pm25, column ='TotalPopD', title ='Mortality at Top 5 PM2.5 Counties', gdf_emis = gdf_emis, species =  target_sp_list)

# --- Map Plot: PM2.5 at Top PM2.5 Counties ---
plot_map_with_fips(top_pm25, column ='TotalPM25', cmap="OrRd_r", title ='PM2.5 at Top 5 PM2.5 Counties', gdf_emis = gdf_emis, species  =  target_sp_list)

# --- Bar Plot: Top 5 Mortality Counties ---
top_mortality.plot(kind='bar', legend=True, figsize=(8, 4))
plt.title("Top 5 Counties by Mortality")
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# --- Map Plot: Mortality at Top Mortality Counties ---
plot_map_with_fips(top_mortality, column ='TotalPopD', title ='Mortality at Top 5 Mortality Counties', gdf_emis = gdf_emis, species =  target_sp_list)

# --- Map Plot: PM2.5 at Top Mortality Counties ---
plot_map_with_fips(top_mortality, column ='TotalPM25', cmap="OrRd_r", title = 'PM2.5 at Top 5 Mortality Counties',gdf_emis = gdf_emis, species =  target_sp_list)


In [ ]:
# --- Select Top Counties ---

for sp in emis_column_list: 
    top_emis = gdf_emis_county.sort_values(by=sp, ascending=False).head(5).set_index('FIPS')

    print(top_emis.head())

    # --- Bar Plot: Top 5 PM2.5 Counties ---
    top_emis.plot(kind='bar', legend=True, figsize=(8, 4))
    plt.title(f"Top 5 Counties by {sp} emission")
    # Place the legend outside the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    #plt.legend(bbox_to_anchor=(1.0, 1.0))

    #plt.ylabel("Total PM2.5")
    plt.tight_layout()
    plt.show()
    plt.close('all')

    # --- Map Plot: Mortality at Top PM2.5 Counties ---
    plot_map_with_fips(top_emis, column =sp, title =f'Top 5 Counties by {sp} emission', gdf_emis = gdf_emis, species =  target_sp_list)


## Analysis begins

# Approach 1

This approach finds the areas with large health impacts and identifies the facilities (emitters) falls in the buffer

In [ ]:


# Plot a spatial map of the top 10 high emitters for each species
for target_sp in target_sp_list:
    # sort the emission data
    gdf_emis = gdf_emis.sort_values(by=target_sp, ascending=False)

    plot_buffer_and_base(gdf_county, 'TotalPopD', gdf_emis, target_sp, number_of_buffer=5)


# Plot a spatial map of the top 3 emitters at the high mortality area
output_dir = f"/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/find_facility/LA_runs/"
# Buffer distance in meters
buffer_distance_list = [1000, 5000, 50000]  # 100 km

gdf_health_county_top = find_top_3_facilities(gdf_county, gdf_emis, buffer_distance_list, target_sp_list, output_dir)


# Bar plot to find the county with high mortality 
fig, ax = plt.subplots(figsize=(15, 5))  # Adjust figure size if needed

gdf_county.sort_values(by = 'TotalPopD', ascending=False, inplace=True)

gdf_county[:5].plot(ax=ax, kind='bar', x= 'FIPS', y ='TotalPopD', width=0.8, xlabel = 'FIPS', ylabel='Total Mortality')  # Plot the bar chart
for p in ax.patches:
    value = np.round(p.get_height(),decimals=0)
    ax.annotate(int(value), (p.get_x() * 1.005, value * 1.005))
plt.show()

################################
# Now, let's look at the ranking 
################################

# Drop the geometry column if it exists
gdf_rank = gdf_health_county_top.copy()

# Loop through each group of bars (one group per column)
for i, col in enumerate(gdf_rank.columns):
    if col != 'geometry' and col != 'FIPS':  # skip geometry if exists
        #compute 
        gdf_rank["rank_" + col]= gdf_rank[col].rank(method='first', ascending=False).astype(int)  # You can also try method='first'
        gdf_rank.drop(columns = col, inplace=True)
    
print(gdf_rank.head())

gdf_rank.set_index('FIPS', inplace=True)

# Make a bar plot to show the high ranking by county-level mortality
fig, ax = plt.subplots(figsize=(18, 6))

gdf_rank.plot(ax=ax, kind='bar', width=0.8, xlabel = 'FIPS', ylabel = 'Ranking by total mortality')  # Plot the bar chart

max_rank = len(gdf_rank)

for i, p in enumerate(ax.patches):
    value = np.round(p.get_height(),decimals=0)
    # Normalize rank to [0.3, 1.0] range for alpha (you can tweak min alpha)
    alpha = 1.0 - (value - 1) / (max_rank - 1)*0.9   # higher rank = more opaque

    p.set_alpha(alpha)
    ax.annotate(int(value), (p.get_x() * 1.005, value * 1.005))

plt.tight_layout()
plt.show()

# Plot a spatial map of top rank
number_of_buffers = 5
for i, col in enumerate(gdf_rank.columns):
    if 'buffer' in col:  
        plot_buffer_and_base(gdf_health_county_top, 'TotalPopD', gdf_rank, col, number_of_buffer =  number_of_buffers, buffer_plot_first=False)


# APPROACH 2

This approach finds the emitters first and then find the health impacts. 

In [ ]:
def save_emitters_and_buffers_separately(gdf_health, gdf_emis, buffer_distance_list, species_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for sp in species_list:
        # Get top 10 facilities for this species
        gdf_top_emitters = gdf_emis.nlargest(10, sp).copy()
        gdf_top_emitters = gdf_top_emitters[['geometry', 'FIPS', 'EIS_ID', sp]]

        # Prepare facility GeoDataFrame (point geometry)
        gdf_facilities = gdf_top_emitters.copy()
        gdf_facilities['FIPS: EIS_ID'] = gdf_facilities['FIPS'].astype(str) + ": " + gdf_facilities['EIS_ID'].astype(str)
        gdf_facilities = gdf_facilities.rename(columns={sp: f'{sp}_emis'})

        # Drop duplicate columns for clarity
        gdf_facilities = gdf_facilities[['geometry', 'FIPS: EIS_ID', f'{sp}_emis']]

        # Save point-based facility data
        facility_out_path = os.path.join(output_dir, f"top10_{sp}_facilities_points.shp")
        gdf_facilities.to_file(facility_out_path)


        # Prepare a list to hold the new records
        buffer_records = []

        for idx, facility in gdf_top_emitters.iterrows():
            facility_id = f"{facility['FIPS']}: {facility['EIS_ID']}"
            emis_val = facility[sp]
            record = {
                'FIPS: EIS_ID': facility_id,
                f'{sp}_emis': emis_val,
            }

            # Compute mortality for each buffer distance and store as separate column
            for buffer_distance in buffer_distance_list:
                buffer_geom = facility.geometry.buffer(buffer_distance)
                affected_grids = gdf_health[gdf_health.intersects(buffer_geom)]
                total_mortality = affected_grids['TotalPopD'].abs().sum()
                col_name = f'mort_{int(buffer_distance / 1000)}_km'
                record[col_name] = total_mortality
                record['geometry'] = buffer_geom  # store buffer geometry

            buffer_records.append(record)

        # Save buffer-based mortality data
        gdf_buffers = gpd.GeoDataFrame(buffer_records, crs=gdf_emis.crs)
        buffer_out_path = os.path.join(output_dir, f"top10_{sp}_buffers_mortality.shp")
        gdf_buffers.to_file(buffer_out_path)

    return gdf_facilities, gdf_buffers

top_emitters, buffer_mortality = save_emitters_and_buffers_separately(gdf_county, gdf_emis, buffer_distance_list, target_sp_list, output_dir)

In [ ]:
def find_top_5_emitters_and_mortality(gdf_health, gdf_emis, buffer_distance_list, species_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for sp in species_list:
        # Get top 10 facilities for this species
        gdf_top_emitters = gdf_emis.nlargest(5, sp).copy()
        gdf_top_emitters = gdf_top_emitters[['geometry', 'FIPS', 'EIS_ID', sp]]
        
        facility_ids = []
        facility_emis = []
        
        for idx, facility in gdf_top_emitters.iterrows():

            facility_ids.append(f"{facility['FIPS']}: {facility['EIS_ID']}")
            facility_emis.append(facility[sp])
            
            for buffer_distance in buffer_distance_list:

                buffer_geoms = []
                buffer_mortalities = []

                facility_buffer = facility.geometry.buffer(buffer_distance)
                buffer_geoms.append(facility_buffer)

                # Find intersecting health areas and sum mortality
                affected_grids = gdf_health[gdf_health.intersects(facility_buffer)]
                total_mortality = affected_grids['TotalPopD'].abs().sum()
                buffer_mortalities.append(total_mortality)

            # Create GeoDataFrame for buffers with mortality
            gdf_buffers = gpd.GeoDataFrame({
                'geometry': buffer_geoms,
                'FIPS: EIS_ID': facility_ids,
                f'{sp}_emis': facility_emis,
                f'total_mortality_{buffer_distance}': buffer_mortalities
            }, geometry='geometry')

        # Save shapefile
        gdf_buffers.to_file(os.path.join(output_dir, f"top5_{sp}_emitter_buffers_mortality.shp"))


        # Start plotting
        fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(14, 5), gridspec_kw={'width_ratios': [2, 1]})
        ax2 = ax1.twinx()  # twin axis for the first plot


        gdf_buffers_sorted = gdf_buffers.sort_values(by=f'{sp}_emis', ascending=False)
        x_labels_emitters = gdf_buffers_sorted['FIPS: EIS_ID'].astype(str)

        # Get top 5 high-mortality areas from gdf_health
        gdf_health_sorted = gdf_health.sort_values(by='TotalPopD', ascending=False).head(5)
        x_labels_health = gdf_health_sorted['FIPS'].astype(int).astype(str)
        mortality_health_top10 = gdf_health_sorted['TotalPopD'].abs().astype(float).tolist()


        print("debug ", x_labels_health, mortality_health_top10)

        # First subplot: Emissions-based analysis
        ax1.bar(x_labels_emitters, gdf_buffers_sorted[f'{sp}_emis'], color='orange', alpha=0.6, label=f'{sp} Emissions')
        ax2.plot(x_labels_emitters, gdf_buffers_sorted[f'total_mortality_{buffer_distance}'], 
                    color='blue', marker='o', label='Mortality')

        ax1.set_xlabel("FIPS: EIS_ID")
        ax1.set_ylabel(f"{sp} Emissions")
        ax2.set_ylabel("Total Mortality in Buffer")
        ax1.set_title(f"Top 10 {sp} Emitters & Buffer Mortality")

        ax1.tick_params(axis='x', rotation=45)
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')

        # Second subplot: Top 10 high-mortality areas
        ax3.bar(x_labels_health, mortality_health_top10, color='steelblue', alpha=0.7)
        ax3.set_xlabel("High-Mortality Area FIPS")
        ax3.set_title("Top 5 Mortality Areas (TotalPopD)")

        plt.suptitle(f"{sp} Emissions vs. Mortality Comparison ({buffer_distance/1000:.0f} km Buffer)")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(output_dir, f"top5_{sp}_mortality_comparison_{buffer_distance}.png"))
        plt.close()

find_top_5_emitters_and_mortality(gdf_county, gdf_emis,buffer_distance_list, target_sp_list, output_dir )

In [ ]:
def find_top_emitters_combined_plot(gdf_health, gdf_emis, buffer_distance_list, species_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for buffer_distance in buffer_distance_list:
        combined_data = {}

        # Loop over species and collect data
        for sp in species_list:
            gdf_top_emitters = gdf_emis.nlargest(10, sp).copy()
            gdf_top_emitters = gdf_top_emitters[['geometry', 'FIPS', 'EIS_ID', sp]]

            for idx, facility in gdf_top_emitters.iterrows():
                facility_id = f"{facility['FIPS']}: {facility['EIS_ID']}"
                emis_val = facility[sp]
                facility_buffer = facility.geometry.buffer(buffer_distance)
                affected_grids = gdf_health[gdf_health.intersects(facility_buffer)]
                mortality = affected_grids['TotalPopD'].abs().sum()

                if facility_id not in combined_data:
                    combined_data[facility_id] = {'mortality': mortality}
                combined_data[facility_id][sp] = emis_val

        # Convert to DataFrame for plotting
        df_combined = pd.DataFrame.from_dict(combined_data, orient='index').fillna(0)

        # Sort by mortality or by one species if preferred
        df_combined = df_combined.sort_values(by='mortality', ascending=False).head(10)

        # Plot
        fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={'width_ratios': [2.5, 1]})
        ax2 = ax1.twinx()

        x = np.arange(len(df_combined))
        width = 0.8 / len(species_list)  # side-by-side bars

        # Plot emissions for each species
        for i, sp in enumerate(species_list):
            emis_vals = df_combined[sp] if sp in df_combined else [0]*len(df_combined)
            ax1.bar(x + i * width, emis_vals, width, label=f'{sp} Emis')

        # Plot mortality as a line on top
        ax2.plot(x + width * (len(species_list)-1) / 2, df_combined['mortality'], 
                 color='black', marker='o', label='Mortality')

        ax1.set_xticks(x + width * (len(species_list)-1) / 2)
        ax1.set_xticklabels(df_combined.index, rotation=45)
        ax1.set_xlabel("FIPS: EIS_ID")
        ax1.set_ylabel("Emissions")
        ax2.set_ylabel("Total Mortality in Buffer")
        ax1.set_title(f"Top Emitters & Buffer Mortality (All Species)")
        ax1.legend(loc='upper left')
        ax2.legend(loc='upper right')

        # Top 5 mortality areas from gdf_health
        gdf_health_sorted = gdf_health.groupby('FIPS')['TotalPopD'].sum().sort_values(ascending=False).head(5)
        ax3.bar(gdf_health_sorted.index.astype(str), gdf_health_sorted.values, color='steelblue', alpha=0.7)
        ax3.set_xlabel("High-Mortality Area FIPS")
        ax3.set_ylabel("Total Mortality in Buffer")
        ax3.set_title("Top 5 Mortality Areas (TotalPopD)")

        plt.suptitle(f"Combined Species Emissions vs. Mortality Comparison ({buffer_distance/1000:.0f} km Buffer)")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(output_dir, f"combined_species_mortality_comparison_{buffer_distance}.png"))
        plt.close()

find_top_emitters_combined_plot(gdf_county, gdf_emis,buffer_distance_list, target_sp_list, output_dir )

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import geopandas as gpd
import matplotlib.patches as mpatches
import numpy as np
from adjustText import adjust_text

def plot_top_facilities_in_map(gdf_health, gdf_emis, buffer_distance_list):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for buffer_distance in buffer_distance_list:
        combined_data = {}

        # Loop over species and collect data
        for sp in species_list:
            gdf_top_emitters = gdf_emis.nlargest(10, sp).copy()
            gdf_top_emitters = gdf_top_emitters[['geometry', 'FIPS', 'EIS_ID', sp]]

            for idx, facility in gdf_top_emitters.iterrows():
                facility_id = f"{facility['FIPS']}: {facility['EIS_ID']}"
                emis_val = facility[sp]
                facility_buffer = facility.geometry.buffer(buffer_distance)
                affected_grids = gdf_health[gdf_health.intersects(facility_buffer)]
                mortality = affected_grids['TotalPopD'].abs().sum()

                if facility_id not in combined_data:
                    combined_data[facility_id] = {'mortality': mortality}
                combined_data[facility_id][sp] = emis_val

    # Dictionary to store total mortality within buffer distance for each facility
    facility_mortality_totals = {}

    # Process each facility to calculate total mortality within buffer_distance
    for idx, facility in gdf_emis.iterrows():
        facility_point = facility.geometry
        facility_buffer = facility_point.buffer(buffer_distance)
        intersecting_areas = gdf_county[gdf_county.intersects(facility_buffer)]
        total_mortality = intersecting_areas['TotalPopD'].abs().sum()
        
        facility_mortality_totals[facility['EIS_ID']] = total_mortality

    # Select top 10 facilities with highest total mortality
    top_10_facilities = sorted(facility_mortality_totals, key=facility_mortality_totals.get, reverse=True)[:10]


    gdf_top_10 = gdf_emis[gdf_emis['EIS_ID'].isin(top_10_facilities)]

    # Using EPSG:3395 (World Mercator) for accurate distance measurements
    desired_crs = 'EPSG:3395'
    gdf_top_10 = gdf_top_10.to_crs(desired_crs)
    gdf_health = gdf_health.to_crs(desired_crs)


    # Create buffers for each facility point
    gdf_buffers = gdf_top_10.copy()
    gdf_buffers['geometry'] = gdf_top_10.geometry.buffer(buffer_distance)


    # Define buffer distance (in meters since we're using a projected CRS)
    buffer_distance = 100000  # 100 km

    # Create a completely new figure and axis
    fig, ax = plt.subplots(figsize=(14, 14))

    # 1. First plot health state areas
    health_plot = gdf_health.plot(column='TotalPopD', 
                                        ax=ax, 
                                        cmap='viridis', 
                                        alpha=0.8,
                                        legend=True,
                                        legend_kwds={'label': 'Total Premature Mortality'})

    # 2. Create and plot buffers manually
    for idx, row in gdf_top_10.iterrows():
        # Create buffer geometry
        buffer_geom = row.geometry.buffer(buffer_distance)
        
        # Create a patch for the buffer
        if hasattr(buffer_geom, 'exterior'):
            # Single polygon case
            xs, ys = buffer_geom.exterior.xy
            ax.fill(xs, ys, alpha=0.3, fc='lightblue', ec='blue', linewidth=1.5)
        else:
            # Multi-polygon case, handle each part
            for geom in buffer_geom.geoms:
                xs, ys = geom.exterior.xy
                ax.fill(xs, ys, alpha=0.3, fc='lightblue', ec='blue', linewidth=1.5)

    # 3. Plot emission points with very explicit styling
    for idx, row in gdf_top_10.iterrows():
        x, y = row.geometry.x, row.geometry.y
        ax.plot(x, y, 'r*', markersize=20, markeredgecolor='black', markeredgewidth=1.5, zorder=5)

    # Create custom legend
    blue_patch = mpatches.Patch(color='lightblue', alpha=0.3, edgecolor='blue', label='100km Buffer')
    point_patch = mpatches.Patch(color='red', label='Emission Points')

    # Add legend
    ax.legend(handles=[blue_patch, point_patch], loc='upper right')

    # Add title and labels
    plt.title(f'Emission Points, {buffer_distance/1000} km Buffers, and Health State Areas', fontsize=16)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)

    # Create a list to hold all text objects
    texts = []

    offsets = {
        0: (20, -20),
        1: (30, -30),
        2: (-50, 20),
        3: (0, -40),
        4: (-40, -10),
        5: (40, 10),
        6: (0, 40),
        7: (-30, -30),
        8: (30, 30),
        9: (-20, 30)
    }


    for rank, facility_id in enumerate(top_10_facilities):
        facility_row = gdf_top_10[gdf_top_10['EIS_ID'] == facility_id].iloc[0]
        x, y = facility_row.geometry.x, facility_row.geometry.y
        dx, dy = offsets.get(rank, (10, 10))
        
        ax.annotate(f"#{rank+1}: {facility_id}", 
                    xy=(x, y),
                    xytext=(dx, dy),
                    textcoords="offset points",
                    fontsize=11,
                    fontweight='bold',
                    bbox=dict(boxstyle="round,pad=0.4", fc="white", ec="red", alpha=0.9),
                    arrowprops=dict(arrowstyle="->", color='gray'))

    # Adjust to prevent overlapping
    adjust_text(texts, ax=ax, arrowprops=dict(arrowstyle='->', color='gray'))

    # Make sure axes aspect is set appropriately
    ax.set_aspect('equal')

    # Show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
def find_top_emitters_combined(gdf_health, gdf_emis, buffer_distance_list, species_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for buffer_distance in buffer_distance_list:
        combined_data = {}

        # Step 1: Collect top 10 emitters by species
        for sp in species_list:
            gdf_top_emitters = gdf_emis.nlargest(10, sp).copy()

            print("gdf_top_emitters ", gdf_top_emitters.head())
            for idx, facility in gdf_top_emitters.iterrows():
                facility_id = f"{facility['FIPS']}: {facility['EIS_ID']}"
                emis_val = facility[sp]
                buffer_geom = facility.geometry.buffer(buffer_distance)
                affected_grids = gdf_health[gdf_health.intersects(buffer_geom)]
                mortality = affected_grids['TotalPopD'].abs().sum()

                if facility_id not in combined_data:
                    combined_data[facility_id] = {'mortality': mortality}

                combined_data[facility_id][sp] = emis_val

        # Convert to DataFrame
        df_combined = pd.DataFrame.from_dict(combined_data, orient='index').fillna(0)

        # Normalize each species’ emission column
        # for sp in species_list:
        #     max_val = df_combined[sp].max()
        #     if max_val > 0:
        #         df_combined[sp] = df_combined[sp] / max_val

        # Sort by mortality
        df_combined = df_combined.sort_values(by='mortality', ascending=False)

        print("df_combined", df_combined.head())

        # Plotting
        fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(18, 6), gridspec_kw={'width_ratios': [2.5, 1]})
        ax2 = ax1.twinx()

        x_labels = df_combined.index.tolist()
        x = np.arange(len(x_labels))
        width = 0.8 / len(species_list)

        # Bar plots for each species (side-by-side normalized bars)
        for i, sp in enumerate(species_list):
            ax1.bar(x + i * width, df_combined[sp], width, label=f'{sp} (norm)')

        # Mortality line (centered)
        ax2.plot(x + width * (len(species_list) - 1) / 2,
                 df_combined['mortality'],
                 color='black', marker='o', label='Mortality')

        ax1.set_xticks(x + width * (len(species_list) - 1) / 2)
        ax1.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=9)
        ax1.set_xlabel("FIPS: EIS_ID")
        ax1.set_ylabel("Emissions in Log scale")
        ax1.set_yscale("log")
        ax2.set_ylabel("Total Mortality in Buffer")
        ax1.set_title("Top 10 Emitters by Species (Normalized) & Buffer Mortality")
        ax1.legend(loc='upper left', bbox_to_anchor=(0.85, 0.9))
        ax2.legend(loc='upper right')

        # Second subplot: Top 5 Mortality Areas from gdf_health
        gdf_health_sorted = gdf_health.groupby('FIPS')['TotalPopD'].sum().sort_values(ascending=False).head(5)
        ax3.bar(gdf_health_sorted.index.astype(str), gdf_health_sorted.values, color='steelblue', alpha=0.7)
        ax3.set_xlabel("High-Mortality Area FIPS")
        ax3.set_ylabel("Total Mortality in Buffer")
        ax3.set_title("Top 5 Mortality Areas (TotalPopD)")

        plt.suptitle(f"Combined Species Emissions vs. Mortality Comparison ({buffer_distance / 1000:.0f} km Buffer)")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(output_dir, f"combined_mortality_comparison_{buffer_distance}.png"))
        plt.close()

# Buffer distance in meters
buffer_distance_list = [1000, 5000, 10000, 30000, 50000]  # 100 km

find_top_emitters_combined(gdf_county, gdf_emis,buffer_distance_list, target_sp_list, output_dir )

In [ ]:
def find_top_emitters_combined_stacked(gdf_health, gdf_emis, buffer_distance_list, species_list, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for buffer_distance in buffer_distance_list:
        combined_data = {}

        for sp in species_list:
            gdf_top_emitters = gdf_emis.nlargest(10, sp).copy()

            for idx, facility in gdf_top_emitters.iterrows():
                facility_id = f"{facility['FIPS']}: {facility['EIS_ID']}"
                emis_val = facility[sp]
                buffer_geom = facility.geometry.buffer(buffer_distance)

                # All facilities within this buffer for this species
                facilities_in_buffer = gdf_emis[gdf_emis.geometry.within(buffer_geom)]
                total_emis_in_buffer = facilities_in_buffer[sp].sum()

                # Mortality from gdf_health in buffer
                affected_grids = gdf_health[gdf_health.intersects(buffer_geom)]
                mortality = affected_grids['TotalPopD'].abs().sum()

                if facility_id not in combined_data:
                    combined_data[facility_id] = {'mortality': mortality}

                combined_data[facility_id][f"{sp}_top"] = emis_val
                combined_data[facility_id][f"{sp}_others"] = max(total_emis_in_buffer - emis_val, 0)

        df_combined = pd.DataFrame.from_dict(combined_data, orient='index').fillna(0)
        df_combined = df_combined.sort_values(by='mortality', ascending=False)

        print("debug ", df_combined)

        # Plotting
        fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(18, 6), gridspec_kw={'width_ratios': [2.5, 1]})
        ax2 = ax1.twinx()

        x_labels = df_combined.index.tolist()
        x = np.arange(len(x_labels))
        width = 0.8 / len(species_list)

        # Stacked bar for each species
        for i, sp in enumerate(species_list):
            top_vals = df_combined[f"{sp}_top"]
            others_vals = df_combined[f"{sp}_others"]

            ax1.bar(x + i * width, top_vals, width, label=f'{sp} (Top)', alpha=0.7)
            ax1.bar(x + i * width, others_vals, width, bottom=top_vals, label=f'{sp} (Others)', alpha=0.4)

        # Mortality line
        ax2.plot(x + width * (len(species_list) - 1) / 2,
                 df_combined['mortality'],
                 color='black', marker='o', label='Mortality')

        ax1.set_xticks(x + width * (len(species_list) - 1) / 2)
        ax1.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=9)
        ax1.set_xlabel("FIPS: EIS_ID")
        ax1.set_ylabel("Emissions in Log scale")
        ax1.set_yscale("log")
        ax2.set_ylabel("Total Mortality in Buffer")
        ax1.set_title("Top 10 Emitters by Species (Stacked) & Buffer Mortality")
        ax1.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
        ax2.legend(loc='upper right')

        # Second subplot: Top 5 Mortality Areas from gdf_health
        gdf_health_sorted = gdf_health.groupby('FIPS')['TotalPopD'].sum().sort_values(ascending=False).head(5)
        ax3.bar(gdf_health_sorted.index.astype(str), gdf_health_sorted.values, color='steelblue', alpha=0.7)
        ax3.set_xlabel("High-Mortality Area FIPS")
        ax3.set_ylabel("Total Mortality in Buffer")
        ax3.set_title("Top 5 Mortality Areas (TotalPopD)")

        plt.suptitle(f"Combined Species Emissions vs. Mortality Comparison ({buffer_distance / 1000:.0f} km Buffer)")
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(output_dir, f"combined_stacked_mortality_comparison_{buffer_distance}.png"))
        plt.close()

buffer_distance_list = [1000, 5000, 10000, 30000, 50000]  # 100 km

find_top_emitters_combined_stacked(gdf_county, gdf_emis, buffer_distance_list, target_sp_list, output_dir)

In [ ]:

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point


buffer_distance = 1000 # 1 km 

# Dictionary to store total mortality within buffer distance for each facility
facility_mortality_totals = {}

# Process each facility to calculate total mortality within buffer_distance
for idx, facility in gdf_emis.iterrows():
    facility_point = facility.geometry
    facility_buffer = facility_point.buffer(buffer_distance)
    intersecting_areas = gdf_county[gdf_county.intersects(facility_buffer)]
    total_mortality = intersecting_areas['TotalPopD'].abs().sum()
    
    facility_mortality_totals[facility['EIS_ID']] = total_mortality

# Select top 10 facilities with highest total mortality
top_10_facilities = sorted(facility_mortality_totals, key=facility_mortality_totals.get, reverse=True)[:10]

# Initialize a dictionary to store cumulative Mortality data for plotting
plot_data = {}

# Define distance bins
distance_bins = np.arange(0, buffer_distance + 1000, 1000)

# Process each of the top 10 facilities
for facility_id in top_10_facilities:
    facility = gdf_emis[gdf_emis['EIS_ID'] == facility_id].iloc[0]
    facility_point = facility.geometry
    cumulative_mortality = []
    
    # Find the health area that contains the facility point
    containing_area = None
    for idx, health_area in gdf_county.iterrows():
        if health_area.geometry.contains(facility_point):
            containing_area = health_area
            break
    
    # Initialize mortality by distance dictionary
    mortality_by_distance = {dist: 0 for dist in distance_bins}
    
    # If a containing area is found, add its mortality to the first bin
    if containing_area is not None:
        # Use built-in abs() function for scalar values
        mortality_by_distance[0] = abs(containing_area['TotalPopD'])
    
    # For each health area, find the minimum distance to the facility
    # and add its mortality to the appropriate bin
    for idx, health_area in gdf_county.iterrows():
        # Skip the containing area as we've already handled it
        if containing_area is not None and idx == containing_area.name:
            continue
            
        # Calculate minimum distance between facility and health area
        distance = facility_point.distance(health_area.geometry)
        
        # Find the appropriate bin
        for bin_edge in sorted(mortality_by_distance.keys()):
            if distance <= bin_edge:
                # Use built-in abs() function for scalar values
                mortality_by_distance[bin_edge] += abs(health_area['TotalPopD'])
                break
    
    # Accumulate mortality for plotting
    running_total = 0
    for dist in sorted(distance_bins):
        running_total += mortality_by_distance[dist]
        cumulative_mortality.append(running_total)
    
    plot_data[facility_id] = cumulative_mortality

# Plotting all facilities in the same plot
plt.figure(figsize=(12, 8))

for facility_id, cumulative_mortality in plot_data.items():
    plt.plot(distance_bins / 1000, cumulative_mortality, label=facility_id)

plt.xlabel('Distance from Facility (km)')
plt.ylabel('Cumulative Total Premature Mortality (absolute value)')
plt.title('Cumulative Total Premature Mortality vs. Distance from Top 10 Facilities')
plt.legend()
plt.grid(True)
plt.ylim(bottom=0)
plt.show()


In [ ]:
gdf_top_10